## Import Spark Libraries

In [1]:
import org.apache.spark.sql
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.mllib.linalg.Vectors.fromML
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.MultilayerPerceptronClassifier
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.util.MLUtils
import org.apache.spark.rdd.RDD
import org.apache.spark.mllib.linalg.DenseVector
import org.apache.spark.ml.feature.{IndexToString, StringIndexer, VectorIndexer}
import org.apache.spark.ml.feature.VectorAssembler

Intitializing Scala interpreter ...

Spark Web UI available at http://46943de61489:4040
SparkContext available as 'sc' (version = 2.4.3, master = local[*], app id = local-1559991775716)
SparkSession available as 'spark'


import org.apache.spark.sql
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.mllib.linalg.Vectors.fromML
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.MultilayerPerceptronClassifier
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.util.MLUtils
import org.apache.spark.rdd.RDD
import org.apache.spark.mllib.linalg.DenseVector
import org.apache.spark.ml.feature.{IndexToString, StringIndexer, VectorIndexer}
import org.apache.spark.ml.feature.VectorAssembler


## Import & Merge EDA DataFrames

In [2]:
// Load wrangled datasets from Phase 2: part1.csv; part2.csv; part3.csv
var creditRiskdf1 = spark.read.option("header","true").csv("_edaDataFrames/creditRiskdf2_1.csv")
var creditRiskdf2 = spark.read.option("header","true").csv("_edaDataFrames/creditRiskdf2_2.csv")
var creditRiskdf3 = spark.read.option("header","true").csv("_edaDataFrames/creditRiskdf2_3.csv")

creditRiskdf1: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 186 more fields]
creditRiskdf2: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 186 more fields]
creditRiskdf3: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 186 more fields]


In [3]:
// Merge Dataframes & Cache
var creditRiskdf1and2 = creditRiskdf1.union(creditRiskdf2)
var creditRiskdf = creditRiskdf1and2.union(creditRiskdf3)
creditRiskdf.cache()

creditRiskdf1and2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 186 more fields]
creditRiskdf: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 186 more fields]
res0: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 186 more fields]


In [4]:
var sumCountOrig = creditRiskdf1.count()+creditRiskdf2.count()+creditRiskdf3.count()
var countMerge = creditRiskdf.count()

if (sumCountOrig == countMerge){
    println("Files Successfully Merged!")
}else{
    println("File Merge Unsuccessful :(")
}

2019-06-08 11:04:24,968 WARN  [Thread-4] util.Utils (Logging.scala:logWarning(66)) - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.
Files Successfully Merged!


sumCountOrig: Long = 35583
countMerge: Long = 35583


In [5]:
// Describe Dataframe
println("DataFrame Size: "+creditRiskdf.columns.length+" columns & "+ creditRiskdf.count()+" rows")

DataFrame Size: 188 columns & 35583 rows


In [6]:
// change datatype of data from String to Float
var creditRiskFeatures = creditRiskdf.columns

for (colName<-creditRiskFeatures){
    creditRiskdf = creditRiskdf.withColumn(
        colName,col(colName).cast("Float"))
}

creditRiskFeatures: Array[String] = Array(EXT_SOURCE_2, EXT_SOURCE_3, DAYS_BIRTH, AMT_CREDIT, AMT_ANNUITY, DAYS_ID_PUBLISH, previous_loans_CNT_PAYMENT_mean, previous_loans_SELLERPLACE_AREA_max, DAYS_LAST_PHONE_CHANGE, DAYS_REGISTRATION, CODE_GENDER_F, REGION_POPULATION_RELATIVE, client_cash_CNT_INSTALMENT_max_mean, bureau_DAYS_CREDIT_mean, previous_loans_NAME_CONTRACT_STATUS_Refused_count_norm, client_installments_NUM_INSTALMENT_VERSION_max_mean, bureau_DAYS_CREDIT_min, previous_loans_NAME_YIELD_GROUP_high_count_norm, previous_loans_RATE_DOWN_PAYMENT_sum, previous_loans_DAYS_FIRST_DUE_min, client_cash_SK_DPD_DEF_sum_max, bureau_AMT_CREDIT_SUM_LIMIT_mean, bureau_DAYS_CREDIT_sum, previous_loans_DAYS_LAST_DUE_1ST_VERSION_max, previous_loans_PRODUCT_COMBINATION_Cash X-Sell: low_count_norm, ...

## Test/Train Split for Modelling

In [16]:
//Dataframe prep.
creditRiskdf = creditRiskdf.withColumnRenamed("TARGET", "label")
creditRiskdf = creditRiskdf.drop("SK_ID_CURR")

creditRiskdf: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 185 more fields]
creditRiskdf: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 185 more fields]


In [17]:
// Create test/train split @ 70/30 ratio
var Array(dfTrain,dfTest) = creditRiskdf.randomSplit(Array(0.7,0.3))
println("Training set count: "+dfTrain.count() + "\n" +
        "Test set count: "+dfTest.count())

Training set count: 24969
Test set count: 10614


dfTrain: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 185 more fields]
dfTest: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 185 more fields]


In [18]:
creditRiskdf.unpersist()
dfTrain.cache()
dfTest.cache()

res7: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 185 more fields]


## Building the Multilayer Perceptron Model

### Instantiate pipeline

In [23]:
// Index labels, adding metadata to the label column.
// Fit on whole dataset to include all labels in index.
var mlpcLabelIndexer = new StringIndexer().setInputCol("label").setOutputCol(
    "indexedLabel").fit(dfTrain)

var features = dfTrain.columns

var Assembler = new VectorAssembler()
    .setInputCols(features).setOutputCol("features")

//var mlpcTrain = Assembler.transform(dfTrain)

mlpcLabelIndexer: org.apache.spark.ml.feature.StringIndexerModel = strIdx_1097ffacc5f7
features: Array[String] = Array(EXT_SOURCE_2, EXT_SOURCE_3, DAYS_BIRTH, AMT_CREDIT, AMT_ANNUITY, DAYS_ID_PUBLISH, previous_loans_CNT_PAYMENT_mean, previous_loans_SELLERPLACE_AREA_max, DAYS_LAST_PHONE_CHANGE, DAYS_REGISTRATION, CODE_GENDER_F, REGION_POPULATION_RELATIVE, client_cash_CNT_INSTALMENT_max_mean, bureau_DAYS_CREDIT_mean, previous_loans_NAME_CONTRACT_STATUS_Refused_count_norm, client_installments_NUM_INSTALMENT_VERSION_max_mean, bureau_DAYS_CREDIT_min, previous_loans_NAME_YIELD_GROUP_high_count_norm, previous_loans_RATE_DOWN_PAYMENT_sum, previous_loans_DAYS_FIRST_DUE_min, client_cash_SK_DPD_DEF_sum_max, bureau_AMT_CREDIT_SUM_LIMIT_mean, bureau_DAYS_CREDIT_sum, previous_loans_DAYS_LAST_DUE_1ST_...

In [ ]:
// Automatically identify categorical features, and index them.
//var mlpcFeatureIndexer = new VectorIndexer()
//  .setInputCol("features")
//  .setOutputCol("indexedFeatures")

// specify layers for the neural network:
// input layer of size 4 (features), two intermediate of size 5 and 4
// and output of size 3 (classes)
//https://machinelearningmastery.com/how-to-configure-the-number-of-layers-and-nodes-in-a-neural-network/

var layers = Array[Int](188, 4, 2)

// create the trainer and set its parameters
var mlpcModel = new MultilayerPerceptronClassifier()
  .setLayers(layers)
  .setBlockSize(128)
  .setTol(0.001)
  .setSeed(123)
  .setMaxIter(200)

In [ ]:
// Convert indexed labels back to original labels.
val mlpcLabelConverter = new IndexToString()
  .setInputCol("prediction")
  .setOutputCol("predictedLabel")
  .setLabels(mlpcLabelIndexer.labels)

// Chain indexers and forest in a Pipeline.
var mlpcPipeline = new Pipeline().setStages(
    Array(mlpcLabelIndexer, Assembler, mlpcModel, mlpcLabelConverter))

// train the model
var mlpcModelTrained = mlpcPipeline.fit(dfTest)

In [44]:
mlpcModelTrained.save("/mlpcModel")

<console>: 44: error: not found: value overwrite

In [66]:
var result = mlpcModelTrained.transform(dfTest)

result: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 191 more fields]


In [63]:
result.printSchema

root
 |-- EXT_SOURCE_2: float (nullable = true)
 |-- EXT_SOURCE_3: float (nullable = true)
 |-- DAYS_BIRTH: float (nullable = true)
 |-- AMT_CREDIT: float (nullable = true)
 |-- AMT_ANNUITY: float (nullable = true)
 |-- DAYS_ID_PUBLISH: float (nullable = true)
 |-- previous_loans_CNT_PAYMENT_mean: float (nullable = true)
 |-- previous_loans_SELLERPLACE_AREA_max: float (nullable = true)
 |-- DAYS_LAST_PHONE_CHANGE: float (nullable = true)
 |-- DAYS_REGISTRATION: float (nullable = true)
 |-- CODE_GENDER_F: float (nullable = true)
 |-- REGION_POPULATION_RELATIVE: float (nullable = true)
 |-- client_cash_CNT_INSTALMENT_max_mean: float (nullable = true)
 |-- bureau_DAYS_CREDIT_mean: float (nullable = true)
 |-- previous_loans_NAME_CONTRACT_STATUS_Refused_count_norm: float (nullable = true)
 |-- client_installments_NUM_INSTALMENT_VERSION_max_mean: float (nullable = true)
 |-- bureau_DAYS_CREDIT_min: float (nullable = true)
 |-- previous_loans_NAME_YIELD_GROUP_high_count_norm: float (nullable

In [62]:
result.count

res36: Long = 10614


In [67]:
val predictionAndLabels = result.select("prediction", "label")

val evaluator = new MulticlassClassificationEvaluator()
  .setMetricName("accuracy")

println(evaluator.evaluate(predictionAndLabels))

2019-06-08 12:00:40,878 ERROR [Executor task launch worker for task 2621] executor.Executor (Logging.scala:logError(91)) - Exception in task 0.0 in stage 891.0 (TID 2621)
org.apache.spark.SparkException: Failed to execute user defined function($anonfun$1: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:40

org.apache.spark.SparkException:  Job aborted due to stage failure: Task 0 in stage 891.0 failed 1 times, most recent failure: Lost task 0.0 in stage 891.0 (TID 2621, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$1: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)

In [ ]:
import or// Index labels, adding metadata to the label column.
// Fit on whole dataset to include all labels in index.
val rfLabelIndexer = new StringIndexer().setInputCol("TARGET").setOutputCol(
    "indexedLabel").fit(c1)

val rfAssembler = new VectorAssembler().setInputCols(features).setOutputCol(
    "features")
val df1 = rfAssembler.transform(c1)

// Automatically identify categorical features, and index them.
// Set maxCategories so features with > 4 distinct values are treated as continuous.
val rfFeatureIndexer = new VectorIndexer()
  .setInputCol("features")
  .setOutputCol("indexedFeatures")
  .setMaxCategories(4).fit(df1)

// Train a RandomForest model.
val modelRf = new RandomForestClassifier()
  .setLabelCol("indexedLabel")
  .setFeaturesCol("indexedFeatures")
  .setNumTrees(20)

// Convert indexed labels back to original labels.
val rfLabelConverter = new IndexToString()
  .setInputCol("prediction")
  .setOutputCol("predictedLabel")
  .setLabels(labelIndexer.labels)

// Chain indexers and forest in a Pipeline.
val rfPipeline = new Pipeline()
  .setStages(Array(rfLabelIndexer, rfFeatureIndexer, modelRf, rfLabelConverter))

// Train model. This also runs the indexers.
val rfModel = pipeline.fit(df1)

// Index labels, adding metadata to the label column.
// Fit on whole dataset to include all labels in index.

var features = dfTest.columns

var Assembler = new VectorAssembler().setInputCols(features).setOutputCol(
    "features")
var mlpcTest = Assembler.transform(dfTest)

mlpcTest = mlpcTest.withColumnRenamed("TARGET", "label")

creditRiskdf.unpersist()
dfTrain.cache()
dfTest.cache()

## Building the GradientBoosted Tree

// Index labels, adding metadata to the label column.
// Fit on whole dataset to include all labels in index.

val features = dfTrain.columns

val rfLabelIndexer = new StringIndexer().setInputCol("TARGET").setOutputCol(
    "indexedLabel").fit(dfTrain)

val rfAssembler = new VectorAssembler().setInputCols(features).setOutputCol(
    "features")
val df1 = rfAssembler.transform(dfTrain)

// specify layers for the neural network:
// input layer of size 4 (features), two intermediate of size 5 and 4
// and output of size 3 (classes)
val layers = Array[Int](4, 5, 4, 3)

// create the trainer and set its parameters
val trainer = new MultilayerPerceptronClassifier()
  .setLayers(layers)
  .setBlockSize(128)
  .setSeed(1234L)
  .setMaxIter(100)

// train the model
val model = trainer.fit(df1)

// compute accuracy on the test set
val result = model.transform(dfTest)
val predictionAndLabels = result.select("prediction", "label")
val evaluator = new MulticlassClassificationEvaluator()
  .setMetricName("accuracy")

println("Test set accuracy = " + evaluator.evaluate(predictionAndLabels))



// Index labels, adding metadata to the label column.
// Fit on whole dataset to include all labels in index.

var features = dfTrain.columns

var Assembler = new VectorAssembler().setInputCols(features).setOutputCol(
    "features")
var mlpcTrain = Assembler.transform(dfTrain)

// specify layers for the neural network:
// input layer of size 4 (features), two intermediate of size 5 and 4
// and output of size 3 (classes)
//https://machinelearningmastery.com/how-to-configure-the-number-of-layers-and-nodes-in-a-neural-network/

var layers = Array[Int](150, 4, 2)

// create the trainer and set its parameters
var mlpcModel = new MultilayerPerceptronClassifier()
  .setLayers(layers)
  .setBlockSize(128)
  .setSeed(1234L)
  .setMaxIter(100)

var mlpcPipeline = new Pipeline().setStages(
    Array(mlpcLabelIndexer, Assembler, mlpcModel))